In [74]:
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')
from fanc_seg_utils.annotation_interface import neuroglancer_utilities,schema_download,schema_upload,statemanager
from fanc_seg_utils.skeletons import catmaid_utilities,skeleton_manipulations,skeletonization
import pandas as pd
import numpy as np
from annotationframeworkclient import FrameworkClient
import nglui
from matplotlib import cm
from nglui.statebuilder import *
from matplotlib import pyplot as plt

In [77]:
client,tokens = neuroglancer_utilities.get_client()
segmentation = 'Dynamic_V1'
seg_url = neuroglancer_utilities.get_cv_path(segmentation)
# Generate Table
table_name = 'Mollie_synapses'
table = schema_download.download_annotation_table(client,table_name)
synapses = schema_download.generate_synapse_table(table,token=tokens['dev'],segmentation_version=segmentation)



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.52it/s]



Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]


Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.86it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.74it/s]



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.02it/s]A

Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  1.67it/s]



Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.16it/s]A


Downloading: 100%|██████████| 1/1 [00:00<00:00,  2.19it/s]

Downloading: 100%|██████████

In [12]:
%run -i 'test.py'

Which segmentation do you want to use?dict_keys(['Image', 'Flat_1', 'FlatV_2', 'Dynamic_V1', 'Dynamic_V2', 'Flat_3', 'Dynamic_V4']) Image


<class 'str'> Image


AttributeError: 'dict_keys' object has no attribute 'values'

In [56]:
# Image layer
img_layer = ImageLayerConfig(name='img',
                             source=client.info.image_source(),
                             )
seg_source = seg_url['url']
seg_layer = SegmentationLayerConfig(name = 'segmentations',
                                        source = seg_source,
                                        selected_ids_column='pt_root_id'
                                        )


points = PointMapper(point_column='pt_position')
anno_layer = AnnotationLayerConfig(name=tag_name,
                                   mapping_rules=points )

sb = StateBuilder(layers=[img_layer, seg_layer,anno_layer],resolution=[4.3, 4.3, 45])
sb.render_state(neurons, return_as='html')



In [48]:
# Image layer
img_layer = ImageLayerConfig(name='img',
                             source=client.info.image_source(),
                             )
# DataFrames 
cmap = cm.hsv(np.linspace(0,1,len(bundles)))
dfs = []
idx = 0
for i in bundles:
    df = MNs[MNs['name'].str.contains(i)]
    df['color'] = cm.colors.rgb2hex(cmap[idx])
    dfs.append(df)
    idx+=1
    

    sbs = []
for i in bundles:
    seg_source = seg_url['url']
    seg_layer = SegmentationLayerConfig(name = 'Bundle' + i,
                                        source = seg_source,
                                        selected_ids_column='pt_root_id',
                                        color_column='color'
                                        )

    sbs.append(StateBuilder(layers=[img_layer, seg_layer],resolution=[4.3, 4.3, 45]))

/Users/brandon/.local/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
# Neuron layers
chained_sb = ChainedStateBuilder(sbs)
chained_sb.render_state(dfs, return_as='html')

In [ ]:
def mn_linkbuilder(segmentation = None):
    client,tokens = neuroglancer_utilities.get_client()

    seg_url = neuroglancer_utilities.get_cv_path(segmentation)
    # Generate Table
    table_name = 'T1MN_somas'
    table = schema_download.download_annotation_table(client,'T1MN_somas')
    MNs = schema_download.generate_soma_table(table,token=tokens['dev'],segmentation_version=segmentation)
    bundles = set([i[3:5] for i in MNs.name])

    # Image layer
    img_layer = ImageLayerConfig(name='img',
                                 source=client.info.image_source(),
                                 )
    # DataFrames 
    cmap = cm.hsv(np.linspace(0,1,len(bundles)))
    dfs = []
    idx = 0
    for i in bundles:
        df = MNs[MNs['name'].str.contains(i)]
        df['color'] = cm.colors.rgb2hex(cmap[idx])
        dfs.append(df)
        idx+=1


        sbs = []
    for i in bundles:
        seg_source = seg_url
        seg_layer = SegmentationLayerConfig(name = 'Bundle' + i,
                                            source = seg_source['url'],
                                            selected_ids_column='pt_root_id',
                                            color_column='color')

        sbs.append(StateBuilder(layers=[img_layer, seg_layer]))

    # Neuron layers
    chained_sb = ChainedStateBuilder(sbs)
    
    
    return(chained_sb.render_state(dfs, return_as='html'))